In [30]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd


%matplotlib inline

In [31]:
data = pd.read_csv('/data/home/krazheva/transactions_without_text.csv')
data_new = data.loc[data['date_key'] > '2021-03-01']
data = data.drop(data.index[119833238:123351243])
data.head()

,date_key,amount,client_inn,partner_inn,client_bik,partner_bik,direction
0,2018-03-01,41000.00,1279230,1,1503,1503,in
1,2018-03-01,482300.00,1224261,13022,1503,1441,in
2,2018-03-01,15.72,1549773,119556,759,759,in
3,2018-03-01,4217742.30,976651,976651,759,759,in
4,2018-03-01,11.77,1413113,1003953,1503,1441,in


Data Clients

In [32]:
clients_first = pd.read_csv('/data/home/krazheva/data_clients.csv')
clients = clients_first.drop(['Клиент_НаличиеЗПпроекта', 'Клиент_ДатаЗавершОтн', 'Клиент_АдресРегистрацииОКАТО', 'Представитель_АдресЖительстваОКАТО', 'Представитель_НачалоДействия', 'Представитель_ОкончаниеДействия'], axis=1)
clients.head()

,inn,Клиент_ДатаРег.,Клиент_ДатаНачалаОтн,Клиент_Сегмент,years
0,1384214,03.09.2015,26.03.2021,МБ,0
1,286507,15.10.2013,25.10.2013,МКП,34
2,648912,12.03.2021,26.03.2021,МБ,0
3,1436001,05.04.2002,11.02.2005,КБ,53
4,918092,19.08.2020,26.03.2021,МКП,0


Fraud

In [33]:
fraud = pd.read_csv('/data/home/krazheva/data_fraud.csv')
fraud.head()

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01


In [34]:
fraud_sort = fraud.sort_values(by=['month_fraud'], ascending=False)
fraud_sort.head()

,inn,month_fraud
0,249847,2021-03-01
120,397979,2021-03-01
680,1894908,2021-03-01
184,1409774,2021-03-01
187,1409646,2021-03-01


In [35]:
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
index1 = fraud_new.iloc[:,0]
index1 = (index1.index.tolist())
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
fraud = fraud.drop(index1)
fraud.head()

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01


In [36]:
fraud = fraud.assign(Fraud_presence = 1)
fraud.head()

,inn,month_fraud,Fraud_presence
0,249847,2021-03-01,1
1,170435,2020-04-01,1
2,1499321,2019-05-01,1
3,533212,2021-03-01,1
4,1623507,2021-03-01,1


Reject

In [37]:
reject = pd.read_csv('/data/home/krazheva/data_reject.csv')
reject = reject.assign(Reject_presence = 1)
reject.head()

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
0,826229,ИП,Отказ от закл.дог.,2017-07-01,1
1,183819,ИП,Отказ от закл.дог.,2017-07-01,1
2,1447227,ИП,Отказ от закл.дог.,2017-07-01,1
3,346763,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1
4,1622547,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1


In [38]:
reject_new = reject.loc[reject['month_reject'] < '2018-03-01']
index2 = reject_new.iloc[:,0]
index2 = (index2.index.tolist())
reject = reject.drop(index2)
reject.head()

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
115,309291,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
166,1753304,Юридическое лицо,Отказ в проведении операции,2019-01-01,1
197,1238773,ИП,Отказ от закл.дог.,2018-12-01,1
229,1209749,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
248,327939,Юридическое лицо,Отказ от закл.дог.,2019-01-01,1


Data Union

In [39]:
step1 = pd.merge(left=data, right=clients, left_on='client_inn', right_on='inn')
step1 = step1.drop(['inn'], axis=1)
step2 = pd.merge_ordered(left=step1, right=fraud, left_on='client_inn', right_on='inn', fill_method=0)
step2 = step2.drop(['inn', 'month_fraud', 'Клиент_ДатаРег.','Клиент_Сегмент', 'Клиент_ДатаНачалаОтн'], axis=1)
step2['Fraud_presence'] = step2['Fraud_presence'].fillna(0)
step3 = pd.merge_ordered(left=step2, right=reject, left_on='client_inn', right_on='inn', fill_method=0)
step3 = step3.drop(['inn', 'РеестрОтказов_ТипУчастника', 'РеестрОтказов_КодОтказаСтрокой', 'month_reject'], axis=1)
step3['Reject_presence'] = step3['Reject_presence'].fillna(0)
final_data = step3.dropna()
final_data = final_data.drop(['direction'], axis=1)
final_data.head()

,date_key,amount,client_inn,partner_inn,client_bik,partner_bik,years,Fraud_presence,Reject_presence
0,2018-03-01,24630.00,1.0,119556.0,1503.0,1503.0,0.0,0.0,0.0
1,2018-03-01,165.81,1.0,119556.0,1503.0,1503.0,0.0,0.0,0.0
2,2018-03-01,531.00,1.0,119556.0,1503.0,1503.0,0.0,0.0,0.0
3,2018-03-01,4.95,1.0,119556.0,759.0,759.0,0.0,0.0,0.0
4,2018-03-01,69.78,1.0,119556.0,759.0,759.0,0.0,0.0,0.0


Divide by year

In [40]:
Year_2018 = final_data[final_data.date_key.str.contains('2018')]
Year_2019 = final_data[final_data.date_key.str.contains('2019')]
Year_2020 = final_data[final_data.date_key.str.contains('2020')]
Year_2021 = final_data[final_data.date_key.str.contains('2021')]

Devide by mounth

Year 2018

In [41]:
Year_2018_P1 = Year_2018[Year_2018.date_key < '2018-08-01']
Year_2018_P2 = Year_2018[Year_2018.date_key >= '2018-08-01']

Year 2019

In [42]:
Year_2019_P1 = Year_2019[Year_2019.date_key < '2019-06-01']
Year_2019_P2 = Year_2019[Year_2019.date_key >= '2019-06-01']

Year 2020

In [43]:
Year_2020_P1 = Year_2020[Year_2020.date_key < '2020-06-01']
Year_2020_P2 = Year_2020[Year_2020.date_key >= '2020-06-01']

Random Forest

Для 3/1 прогон #1

In [44]:
from sklearn.model_selection import  learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [45]:
param_grid = {
    'max_depth': [10, 100, 1000]
}

In [46]:
rf = RandomForestClassifier(random_state = 1)

In [47]:
train = pd.concat([Year_2018_P1, Year_2018_P2, Year_2019_P1], ignore_index=True)
train = train.drop(['date_key'], axis=1)
train_data = train.drop(['Fraud_presence', 'Reject_presence'], axis=1)
train_labels_1 = train['Fraud_presence']

In [ ]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_data, train_labels_1)

/data/home/krazheva/.conda/envs/py3.5/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
CV_rfc.best_params_

In [ ]:
rf_best_param=RandomForestClassifier(random_state = 1, max_depth=???)

In [ ]:
rf_best_param.fit(train_data, train_labels_1)

In [ ]:
test_data = Year_2019_P2.drop(['Fraud_presence', 'Reject_presence', 'date_key'], axis=1)
test_data.head()

In [ ]:
predictions_rf = rf.predict(test_data)

In [ ]:
test_labels = Year_2019_P2['Fraud_presence']

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, log_loss
print("Precision:", precision_score(test_labels, predictions_rf))
print("Recall: ", recall_score(test_labels, predictions_rf))
print("Accuracy: ", accuracy_score(test_labels, predictions_rf))
print("F1-score: ", f1_score(test_labels, predictions_rf))
print("Log_loss: ", log_loss(test_labels, predictions_rf))

----------------------------------

Для 3/1 прогон #2

In [ ]:
train1 = pd.concat([Year_2018_P2, Year_2019_P1, Year_2019_P2], ignore_index=True)
train1 = train1.drop(['date_key'], axis=1)
train_data1 = train1.drop(['Fraud_presence', 'Reject_presence'], axis=1)
train_labels1 = train1['Fraud_presence']
test_data1 = Year_2020_P1.drop(['Fraud_presence', 'Reject_presence', 'date_key'], axis=1)
test_labels1 = Year_2020_P1['Fraud_presence']

In [ ]:
CV_rfc_3_1_2 = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rfc_3_1_2.fit(train_data, train_labels_1)

In [ ]:
CV_rfc_3_1_2.best_params_

In [ ]:
rf_best_param_3_1_2=RandomForestClassifier(random_state = 1, max_depth=???)

In [ ]:
rf_best_param_3_1_2.fit(train_data1, train_labels1)

In [ ]:
predictions_rf_3_1_2 = rf.predict(test_data1)

In [ ]:
print("Precision:", precision_score(test_labels1, predictions_rf_3_1_2))
print("Recall: ", recall_score(test_labels1, predictions_rf_3_1_2))
print("Accuracy: ", accuracy_score(test_labels1, predictions_rf_3_1_2))
print("F1-score: ", f1_score(test_labels1, predictions_rf_3_1_2))
print("Log_loss: ", log_loss(test_labels1, predictions_rf_3_1_2))